## Setup & read in data

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt   

In [ ]:
#this is the data including imputed values for test results etc 
all_data = pd.read_csv('imputed_all_data.csv')
parse_dates=['admission_date_structured']
labels = pd.read_csv('disch_los_k_modes_6.csv')
combined_clean = pd.read_csv('combined_clean.csv')


In [ ]:
orig_data_and_labels_comorb = pd.merge(all_data, labels,
                           how='right', on='id')

In [ ]:
orig_data_and_labels_comorb.columns

Index(['Unnamed: 0', 'id', 'Height', 'Weight', 'smoking_structured', 'age',
       'blood_sugar_d1_min', 'blood_sugar_d1_max', 'blood_sugar_d3_min',
       'blood_sugar_d3_max', 'SystolicBP_d1_min', 'SystolicBP_d1_max',
       'SystolicBP_d3_min', 'SystolicBP_d3_max', 'DiastolicBP_d1_min',
       'DiastolicBP_d1_max', 'DiastolicBP_d3_min', 'DiastolicBP_d3_max',
       'Pulse_d1_average', 'Pulse_d3_average', 'Oxygensaturation',
       'Bloodoxygensaturation_d3_average', 'whitebloodcell', 'Neutrophilcount',
       'Lymphocytecount', 'Hemoglobin', 'Plateletscount', 'C-reactiveprotein',
       'ALTkl;', 'AST', 'TotalBilirubin', 'Directbilirubin', 'Bloodsugar',
       'CystatinC', 'Serumsodium', 'Lactatedehydrogenase', 'Creatinine',
       'Serumbloodureanitrogen', 'B-typenatriureticpeptide',
       'HypersensitivetroponinI', 'D-dimer', 'Albumin',
       'AlbumintoGlobulinRatio', 'Male', 'Consciousness', 'Temperature',
       'Respiratoryrate', 'Redcelldistributionwidth', 'SystolicBP',
    

In [ ]:
#remove comorbidities 
orig_data_and_labels = orig_data_and_labels_comorb.drop(columns=['Unnamed: 0','morbidity_Diabetes','morbidity_COPD','morbidity_Hypertension','morbidity_Heartdisease','morbidity_Renaldisease','morbidity_Tumor','morbidity_Metabolicdisorders','morbidity_Respiratorydiseases'])

In [ ]:
orig_data_and_labels

,id,Height,Weight,smoking_structured,age,blood_sugar_d1_min,blood_sugar_d1_max,blood_sugar_d3_min,blood_sugar_d3_max,SystolicBP_d1_min,...,AlbumintoGlobulinRatio,Male,Consciousness,Temperature,Respiratoryrate,Redcelldistributionwidth,SystolicBP,DiastolicBP,Lymphocyte(%),cluster
0,100251,155.00000,60.0,0,62.0,4.660000,4.660000,4.925083,5.956664,124.000000,...,1.382490,False,1.0,36.6,20.0,12.1,134.500000,68.500000,23.384615,3
1,101019,151.00000,63.0,0,72.0,5.300000,5.300000,3.902008,4.507097,116.000000,...,1.129046,False,1.0,36.3,23.0,13.6,135.000000,80.000000,32.424242,1
2,102696,169.00000,80.0,0,63.0,5.190000,5.190000,5.538094,7.640623,119.000000,...,1.367413,True,1.0,36.6,19.0,13.4,129.000000,91.000000,32.266155,1
3,101588,161.00000,57.0,0,39.0,4.939551,4.895466,4.694930,4.862858,110.000000,...,1.125873,False,1.0,36.6,22.0,12.5,122.000000,74.500000,25.101747,1
4,100358,164.75961,65.0,0,62.0,5.793085,6.349398,5.576803,5.064091,115.000000,...,1.006937,True,1.0,37.5,24.0,11.9,122.500000,80.000000,15.538462,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792,101460,172.00000,72.0,0,49.0,4.340000,4.340000,5.230405,3.933495,116.061066,...,1.560000,True,0.0,36.8,24.0,12.2,100.326822,66.389026,32.307692,1
2793,100047,172.00000,75.0,0,45.0,4.091314,7.067830,4.062333,4.329303,109.644809,...,1.263107,True,1.0,36.8,22.0,11.4,101.593761,78.039515,18.589744,1
2794,103038,174.00000,72.0,0,56.0,4.330000,4.330000,5.489082,5.853361,112.000000,...,1.460000,True,1.0,36.5,26.0,13.1,124.000000,76.000000,37.792496,3
2795,100889,164.75961,80.0,0,74.0,4.040000,4.040000,4.339569,4.426759,119.000000,...,1.150000,True,1.0,36.5,26.0,12.5,139.500000,100.500000,14.313725,1


### Predicting binary severity 

We want to convert the cluster labels to only use three labels: we leave the two largest cluster labels as they are, since these contain the non-severe cases and are significantly larger than all other clusters. We then set the remaining individuals to be part of the same cluster. Then, these will be separated out and we can then look at these individuals in more detail later as they are more interesting. 

In [ ]:
# value_counts_list = orig_data_and_labels.cluster.value_counts().index.to_list()
# large_cluster = value_counts_list[0]

Here we redefine the clusters into 2 categories: severe, and non-severe based on the severity of the clusters. 

In [ ]:
#for 6_kmodes
limited_clusters = True
if(limited_clusters):
    data_and_labels = orig_data_and_labels.copy()
    #group clusters together into non-severe, medium, severe
    data_and_labels['cluster'] = orig_data_and_labels['cluster'].apply(
        lambda x: 'non-severe' if (x==0 or x==3 or x==4 or x==5 or x==1 or x==3) else x)
#     data_and_labels['cluster'] = data_and_labels['cluster'].apply(
#         lambda x: 'medium' if (x==0) else x)    
    data_and_labels['cluster'] = data_and_labels['cluster'].apply(
        lambda x: 'severe' if (x==2) else x)            

In [ ]:
data_and_labels.cluster.value_counts()

non-severe    2700
severe          97
Name: cluster, dtype: int64

In [ ]:
non_severe = data_and_labels[data_and_labels.cluster=='non-severe']
severe = data_and_labels[data_and_labels.cluster=='severe']

In [ ]:
death_ICU = combined_clean[['id','death','ICU']]

In [ ]:
severe_death_ICU = pd.merge(severe, death_ICU, how='inner', on='id')
non_severe_death_ICU = pd.merge(non_severe, death_ICU, how='inner', on='id')

In [ ]:
severe_death_ICU

,id,Height,Weight,smoking_structured,age,blood_sugar_d1_min,blood_sugar_d1_max,blood_sugar_d3_min,blood_sugar_d3_max,SystolicBP_d1_min,...,Consciousness,Temperature,Respiratoryrate,Redcelldistributionwidth,SystolicBP,DiastolicBP,Lymphocyte(%),cluster,death,ICU
0,101287,160.00000,60.000000,0,58.0,4.773361,5.972382,4.966484,4.477538,132.000000,...,1.0,36.6,22.0,13.800000,136.000000,91.000000,10.510204,severe,True,True
1,102534,164.75961,65.029389,0,88.0,5.970000,5.970000,5.150096,5.364182,109.000000,...,1.0,36.5,20.0,13.900000,136.500000,70.000000,15.866667,severe,False,True
2,102488,170.00000,82.000000,0,72.0,6.250000,6.250000,4.574277,7.494359,112.000000,...,1.0,36.8,19.0,13.400000,131.000000,75.500000,9.523810,severe,False,True
3,100191,155.00000,76.000000,0,62.0,8.830000,8.830000,3.977883,6.355626,73.000000,...,1.0,36.4,17.0,13.200000,112.500000,75.500000,2.668622,severe,True,True
4,100514,164.75961,73.000000,0,25.0,7.200000,7.200000,6.860000,6.860000,149.000000,...,0.0,36.5,22.0,14.400000,175.000000,119.000000,7.538462,severe,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,100843,164.75961,75.000000,0,49.0,4.438236,4.782497,4.129095,6.717860,119.000000,...,1.0,36.5,24.0,13.400000,124.500000,80.000000,8.241758,severe,False,True
93,100692,164.75961,65.029389,0,78.0,6.090000,6.340000,3.997205,3.956286,93.125607,...,1.0,37.2,19.0,12.800000,106.047496,77.620179,5.384615,severe,True,True
94,100181,164.75961,65.029389,0,75.0,4.028059,6.916851,5.744956,5.496064,99.634251,...,1.0,37.4,22.0,14.225441,111.273004,69.780542,33.269486,severe,True,True
95,100338,164.75961,65.029389,0,62.0,4.026961,4.746516,5.471523,5.539986,128.000000,...,0.0,36.9,32.0,12.000000,134.000000,87.000000,3.975155,severe,True,False


In [ ]:
print('percentage non-severe death:')
print((len(non_severe_death_ICU[non_severe_death_ICU['death']==True]))/(len(non_severe_death_ICU))*100)
print('percentage non-severe ICU:')
print((len(non_severe_death_ICU[non_severe_death_ICU['ICU']==True]))/(len(non_severe_death_ICU))*100)

percentage non-severe death:
0.25925925925925924
percentage non-severe ICU:
1.3703703703703705


In [ ]:
print('percentage severe death:')
print((len(severe_death_ICU[severe_death_ICU['death']==True]))/(len(severe_death_ICU))*100)
print('percentage severe ICU:')
print((len(severe_death_ICU[severe_death_ICU['ICU']==True]))/(len(severe_death_ICU))*100)

percentage severe death:
63.91752577319587
percentage severe ICU:
88.65979381443299


In [ ]:
from sklearn import preprocessing, model_selection

def preprocess(all_data):
#     Split the dataset into features and labels (clusters) - so we can normalise the features but not the labels
    all_X = all_data.iloc[:,2:len(all_data.columns)-1]
    all_y = all_data['cluster']
    
    # normalise the data so we have unit variance and mean 0 using built-in preprocessing method in sklearn
    scaler = preprocessing.StandardScaler()
    all_X= pd.DataFrame(scaler.fit_transform(all_X),columns=all_X.columns)
    
#     reset the indexes otherwise it was breaking
    all_X= all_X.reset_index()
    all_y = all_y.reset_index()
    all_X = all_X.drop(columns=['index'])
    all_y = all_y.drop(columns=['index'])
    
    #prepare data for training a model by splitting into training and testing data 
    return (all_X, all_y)

In [ ]:
all_X, all_y = preprocess(data_and_labels)

Let's see if we can identify the types of poor prognoses of the at-risk individuals

## Classification setup

In [ ]:
#Set the class names
class_names = ['severe','non-severe']

### setting up performance metrics calculations

In [ ]:
from sklearn import metrics
#method to compute performance metrics 
def performance_metrics(y_true, y_pred):
    #create a confusion matrix from results 
    cnf_matrix  = metrics.confusion_matrix(y_true, y_pred, labels=class_names, sample_weight=None)
#     # average over all classes to find overall fp,fn,tp,tn
#     FP = np.sum(cnf_matrix, axis=0) - np.diag(cnf_matrix)
#     FN = np.sum(cnf_matrix, axis=1) - np.diag(cnf_matrix)
#     TP = np.diag(cnf_matrix)
#     TN = np.sum(cnf_matrix) - (FP + FN + TP)
        
    #find out if each predicted label is right 
    correct_labels = 0
    for i, true_label in enumerate(y_true):
        if (true_label == y_pred[i]):
            correct_labels += 1

    #calculate overall accuracy
#     acc = np.round(correct_labels/y_pred.shape[0],2)
        
    # tp rate is the same as recall 
#     recall = np.sum(TP)/(np.sum(TP)+np.sum(FN))
#     FP_rate = np.sum(FP)/(np.sum(FP)+np.sum(TN))  
#     precision = np.sum(TP)/(np.sum(TP)+np.sum(FP)) 
#     f_measure = (2*precision*recall)/(precision + recall) 

    metrics_dict = metrics.classification_report(y_true,y_pred,output_dict=True)
    avg_metrics = metrics_dict['macro avg']
    
    recall = avg_metrics['recall']
    precision = avg_metrics['precision']
    f_measure = avg_metrics['f1-score']
    acc = metrics_dict['accuracy']
    
    print(metrics.classification_report(y_true,y_pred))
    
    return (acc, recall, precision, f_measure,
            np.round(cnf_matrix/cnf_matrix.sum(axis=1), 2))


In [ ]:
import seaborn as sn

#method to plot the confusion matrix 
def plot_confusion_matrix(confusion_matrix, class_names):
    conf_matrix = pd.DataFrame(confusion_matrix, index = [i for i in class_names], columns = [i for i in class_names])
    plt.figure()
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(conf_matrix, annot=True, annot_kws={"size": 16}, cmap="Blues")
    plt.show()

## k-fold cross validation

Here we use k-fold cross validation to evaluate the effectiveness of our models, where each 'fold' results in a slightly different train/test split, allowing us to see how the model beaves with different data. We can compute various metrics about the model performance, and from these results, we can choose the most effective classification model.

In [ ]:
from sklearn import model_selection 
from sklearn import linear_model
from sklearn import ensemble

def cross_val(all_X, all_y, class_names, downsampling, upsampling, down_prop, up_prop, clf, printing):
    print('cross-validating...')
    k_val = 3
    #use 3 splits 
    k_fold = model_selection.KFold(n_splits=k_val)
    strat_k_fold = model_selection.StratifiedKFold(n_splits=k_val, shuffle=True)

    #create empty arrays to store the results from each fold
    #we can then average these to get the overall classifications and performance 
    accuracies = np.empty(k_val)
    tp_rates = np.empty(k_val)
    precisions = np.empty(k_val)
    f_measures = np.empty(k_val)

    count = 0

    #split training data into training and validation sets
    for train_indices, validation_indices in strat_k_fold.split(all_X, all_y):
    #     print(X_train.index)
        X_training = all_X.iloc[train_indices]
        y_training = all_y.iloc[train_indices]
        X_validate = all_X.iloc[validation_indices]
        y_validate = all_y.iloc[validation_indices]
        
        # upsample the training data in each fold, if upsample = true
        #with stratified k fold, better not to upsample, as it preserves percentage of samples for each class 
#         print('value counts: ', y_training.value_counts())

        if(downsampling):
#             print('downsampling...')
            X_training, y_training = downsample(X_training, y_training, class_names, down_prop)
            if (printing):
                print('value counts: ', y_training.value_counts())

        if (upsampling):
#             print('upsampling...')
            X_training, y_training = upsample(X_training, y_training, class_names, up_prop)
            if (printing):
                print('value counts:', y_training.value_counts())
            
        # fit classifier
        clf.fit(X_training, y_training.values.ravel())
        
        #predict 
        y_predicted = clf.predict(X_validate)
        y_true_val = y_validate.cluster
        
        #if we are using the decision tree classifier we may want to export the tree to look at it 
        from sklearn import tree

        export_tree = False
        if(export_tree):
            for tree_in_forest in clf.estimators_:
                if(export_tree):
                    tree.export_graphviz(tree_in_forest, out_file="rotated_tree_eg.dot",filled = True, 
                                         feature_names = X_training.columns, class_names=class_names,
                                         rotate = True)
        if (printing):
            print(np.unique(y_predicted, return_counts=True))

        #see how validation set performs 
        acc, recall, precision, f_measure, confusion_matrix = performance_metrics(y_true_val.values, y_predicted)

        if (printing):
            #print performance results for each fold
            print("accuracy: ", acc)
            print("true positive rate / recall: ", recall)
            print("precision: ", precision)
            print("f measure: ", f_measure)
            plot_confusion_matrix(confusion_matrix, class_names)
            print("----------")

        # store info for each fold 
        accuracies[count] = acc
        tp_rates[count] = recall
        precisions[count] = precision
        f_measures[count] = f_measure
        count+=1
        
        
    #average scores
    cv_acc = np.mean(accuracies)
    cv_tp = np.mean(tp_rates)
    cv_precision = np.mean(precisions)
    cv_f_measure = np.mean(f_measures)


    print("overall average performance:")
    print("accuracy: ", cv_acc)
    print("true positive rate/recall: ", cv_tp)
    print("precision: ", cv_precision)
    print("f measure: ", cv_f_measure)
    
    return X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure


Method to perform grid search to find the best parameters for the random forest classifier, uses cross-val


Had to implement this manually due to library constraints and difficulties with upsampling properly inside gridsearch

In [ ]:
from sklearn import model_selection, pipeline, ensemble

def grid_search_rand_forest(data_X, data_y,class_names, downsampling, upsampling, down_prop, up_prop):

    best_f = 0.0
    
    # gridsearch for parameter optimisation
    param_grid = { 
        'n_estimators': [5, 10, 15, 20, 30, 40, 60, 80],
        'max_depth': [5, 10, 15, None],
        'max_features': ['auto', 'sqrt', 'log2'],
        'min_samples_split': [2, 5, 10],
        'criterion' :['gini', 'entropy']}
    
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for max_features in param_grid['max_features']:
                for min_samples_split in param_grid['min_samples_split']:
                    for criterion in param_grid['criterion']:
                        clf = ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                                               min_samples_split=min_samples_split, criterion=criterion)

                        X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, f_measure = cross_val(data_X, data_y, class_names, downsampling, upsampling, down_prop, up_prop, clf, printing=False)

                        if(f_measure > best_f):
                            best_f = f_measure
                            best_clf = clf
    
    # Create grid search object
#     clf = model_selection.GridSearchCV(estimator=ensemble.RandomForestClassifier(), param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

#     best_clf = clf.fit(data_X, data_y.values.ravel())

#     gridsearch_results = pd.DataFrame(best_clf.cv_results_)

    #sort according to overall rank score to find best params
#     best_params_found = gridsearch_results.sort_values('rank_test_score').iloc[0]
    print(best_clf)
    
    return best_clf

## Rebalancing classes

### Downsampling

To address class imbalance by removing some examples in most numerous classes

In [ ]:
def downsample(train_X, train_y, class_names, down_prop):
    
    train_data_and_labels = train_X.copy()
    train_data_and_labels['cluster'] = train_y
    
    #find the largest class and the number of samples in training
    value_counts = train_data_and_labels.cluster.value_counts()
    max_val = max(value_counts)

    most_numerous =  train_data_and_labels.cluster.value_counts().index[0]
    big_cluster = train_data_and_labels[train_data_and_labels['cluster']==most_numerous]
    new_train = train_data_and_labels[train_data_and_labels.cluster!=most_numerous]
    #remove more individuals in largest class if limited classes, as class imbalance is bigger here
    reduced_cluster = big_cluster.sample(replace=False, n=int(len(big_cluster)*down_prop), random_state=1)
    train_down = new_train.append(reduced_cluster)
    y_train_down = train_down['cluster']
    X_train_down = train_down.iloc[:,0:len(train_down.columns)-1]        
    
    return X_train_down, y_train_down

### Upsampling 

This improves the prediction accuracy for the classes containing fewer individuals.

In [ ]:
#upsample the least numerous classes
def upsample(train_X, train_y, class_names, up_prop):
    
    train_data_and_labels = train_X.copy()
    train_data_and_labels['cluster'] = train_y

    #find the largest class and the number of samples in training
    value_counts = train_data_and_labels.cluster.value_counts()
    max_val = max(value_counts)
        
    #upsampling all the classes in training data 
    for cluster in class_names:
        cluster_vals = train_data_and_labels[train_data_and_labels.cluster==cluster]
        num_in_cluster = len(cluster_vals.index)
        num_extra_samples = int(up_prop*(max_val - num_in_cluster))
        new_samples = cluster_vals.sample(replace=True, n=num_extra_samples, random_state=1)
        train_data_and_labels = train_data_and_labels.append(new_samples)
            
#     print(train_data_and_labels.cluster.value_counts())
    
    X_train_up = train_data_and_labels.iloc[:,0:len(train_data_and_labels.columns)-1]
    y_train_up = train_data_and_labels['cluster']
    
    return X_train_up, y_train_up

Set parameters

In [ ]:
#need to do gridsearch 
#for binary kmeans
# clf = grid_search_rand_forest(all_X, all_y, class_names, downsampling=True, upsampling=True, down_prop=0.6, up_prop=0.8,)
# clf= ensemble.RandomForestClassifier(criterion='gini', max_features='sqrt',
#                        min_samples_split=10, n_estimators=40)

#for severe = cluster 2 only 
# clf = grid_search_rand_forest(all_X, all_y, class_names, downsampling=True, upsampling=True, down_prop=0.6, up_prop=0.8)
clf= ensemble.RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=5,
                       n_estimators=15)

We now have a new dataframe containing a more equal distribution of each class to work with. 

Now we perform the k-fold cross validation on data. Since our dataset is small, the predictions can vary a lot based on which data is used for training/testing, which is why k-fold cross-val is useful to get overall prediction metrics. 
If upsampling is true then the least numerous classes will be upsampled.

#### Binaryclass:

In [ ]:
class_names = ['non-severe','severe'] 
X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure = cross_val(all_X, all_y, class_names, downsampling=True, upsampling=True, down_prop=0.6, up_prop=0.8, clf=clf, printing=False)

cross-validating...
              precision    recall  f1-score   support

  non-severe       0.97      0.99      0.98       900
      severe       0.56      0.30      0.39        33

    accuracy                           0.97       933
   macro avg       0.77      0.65      0.69       933
weighted avg       0.96      0.97      0.96       933

              precision    recall  f1-score   support

  non-severe       0.98      0.99      0.99       900
      severe       0.60      0.47      0.53        32

    accuracy                           0.97       932
   macro avg       0.79      0.73      0.76       932
weighted avg       0.97      0.97      0.97       932

              precision    recall  f1-score   support

  non-severe       0.98      0.99      0.98       900
      severe       0.60      0.38      0.46        32

    accuracy                           0.97       932
   macro avg       0.79      0.68      0.72       932
weighted avg       0.97      0.97      0.97       932
